In [9]:
import asyncio
import python_weather
from datetime import datetime
import logging

logger = logging.getLogger(__name__)
WEATHER_LOCATION = "New York City"  # or set from env/config

async def get_real_weather():
    async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
        try:
            weather = await client.get(WEATHER_LOCATION)

            precipitation_chance = 0
            now = datetime.now().hour

            for daily in weather:
                for hourly in daily:
                    if hourly.time.hour >= now:
                        # hourly.chances_of_rain and chances_of_snow are available
                        precipitation_chance = max(hourly.chances_of_rain, hourly.chances_of_snow)
                        break
                if precipitation_chance:
                    break

            return {
                "temperature": int(weather.current.temperature),
                "precipitation": precipitation_chance,
                "conditions": str(weather.current.description)
            }
        except Exception as e:
            logger.error(f"Could not fetch real weather: {e}")
            return {
                "temperature": 0,
                "precipitation": 0,
                "conditions": "Failed to fetch weather data"
            }
get_real_weather()

<coroutine object get_real_weather at 0x10831b780>

In [12]:
import aiohttp
import asyncio

async def get_met_weather(lat=40.7128, lon=-74.0060):
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(url, headers=headers) as resp:
            data = await resp.json()
            current = data["properties"]["timeseries"][0]
            return {
                "time": current["time"],
                "temperature": current["data"]["instant"]["details"]["air_temperature"],
                "wind": current["data"]["instant"]["details"]["wind_speed"]
            }

# In Jupyter:
import nest_asyncio
nest_asyncio.apply()

# Then run:
weather = await get_met_weather()
weather


{'time': '2025-08-17T17:00:00Z', 'temperature': 32.6, 'wind': 4.5}

In [10]:
# Run in a Jupyter cell
daily_forecasts = await get_real_weather()
daily_forecasts

Could not fetch real weather: 'Forecast' object has no attribute 'current'


{'temperature': 0,
 'precipitation': 0,
 'conditions': 'Failed to fetch weather data'}

<coroutine object get_real_weather at 0x110ddd000>